In [2]:
# %pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

**Treinamento do Modelo**

In [ ]:
!pwd

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.yaml')  # build a new model from scratch
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='data.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set

In [6]:
# !pip install supervision ultralytics

**Detecção em Vídeo**

In [8]:
!echo pwd ..

pwd ..


In [ ]:
import numpy as np
import supervision as sv
from ultralytics import YOLO

model = YOLO('runs/detect/train/weights/best.pt')
tracker = sv.ByteTrack()
annotator = sv.RoundBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

sv.process_video(
    source_path="11.2.mp4",
    target_path="resultIDTraceRo.mp4",
    callback=callback
)